# Autosuggestion - Predicting the next word using RNN

Adapted from: https://medium.com/towards-artificial-intelligence/sentence-prediction-using-word-level-lstm-text-generator-language-modeling-using-rnn-a80c4cda5b40

### Approach

- Test with bigger dataset (17000 sentences) vs smaller dataset (1700 sentences)
    - Goal: to identify if a larger training set improves prediction accuracy
    - Bigger dataset: sample of Enron email vs smaller dataset: personal Acronis email made up of mostly newsletters
- Test with long query (>=3 words) short query (1 word)
    - Goal: to identify if there is a gap in the RNN model in predicting the next word

### Steps

1. Get sentence tokens and clean sentences
2. Remove low frequency words and generate training sequences (4 words)
3. Train model and predict

### Setup

- Python version: Python 3.7.4
- Set up virtual environment
    - pip install --user virtualenv
    - virtualenv tensorflow-gpu
    - tensorflow-gpu\Scripts\activate
- Add Jupyter Notebook to virtual environment
    - pip install ipykernel
    - python -m ipykernel install --name=tensorflow-gpu
- Set up Tensorflow 2.1 GPU version
    - https://www.tensorflow.org/install/gpu
    - pip install tensorflow==2.1 
    - CUDA 10.1, cuDNN 7.6.5
- pip install pandas

## Pre-processing

Create txt file to save sentences

In [5]:
import pandas as pd
df = pd.read_csv('enron_emails_clean_2500.csv')
df.fillna('').dropna()
print(df.shape)
df.head()

(17256, 2)


,msg_sentence,msg_clean
0,Here is our forecast,here is our forecast
1,Traveling to have a business meeting takes the...,traveling to have a business meeting takes the...
2,Especially if you have to prepare a presentation.,especially if you have to prepare a presentation
3,I would suggest holding the business plan meet...,i would suggest holding the business plan meet...
4,I would even try and get some honest opinions ...,i would even try and get some honest opinions ...


In [6]:
import numpy as np
np.savetxt(r'enron_train', df.msg_clean.values, fmt='%s')

In [ ]:
f = open("enron_train", "r")
if f.mode == 'r':
    content = f.read()
    print(content)

Data cleaning

In [8]:
import re
# Cleaning the data
clean_data = []
for text in content.splitlines():
    text = text.replace('_', ' ')
    a = re.sub(r'[^a-zA-z ]+', '', text).strip()
    if len(a)>0:
        clean_data.append(a)
    else:
        None

In [9]:
# Removing the lines which are to short or to long
short_data = []
for line in clean_data:
    if 2 <= len(line.split()) <= 200:
        short_data.append(line)
    else:
        None

In [10]:
# Counting the appearnce of each word in the corpus also calculates the number of unique words also
word2count = {}
total_words = 0
for text in short_data:
    for word in text.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
        total_words += 1
print(total_words)

352973


In [11]:
# creating a list that will only contain the words that appear more than 15 times
word15 = []
threshold = 15
for word, count in word2count.items():
    if count >= threshold:
        if len(word) > 1:
            word15.append(word)
print(len(word15))

2614


In [12]:
# Removing the words from each string which appear less than 15 times
data_15 = []
for line in short_data:
    str1=''
    for word in line.split():
        if word in word15:
            str1 = " ".join((str1, word))
    data_15.append(str1.lstrip())

In [13]:
# Removing the lines which are to short or to long after removing the unnecssary words.     
short_data_consize = []
for line in data_15:
    if 3 <= len(line.split()) <= 200:
        short_data_consize.append(line)
    else:
        None

In [14]:
#defining a function to save data
def write_txt(name, data):
    file1 = open("{0}.txt".format(name),"w") 
    for line in data:
        file1.writelines(line) 
        file1.writelines('\n') 
    file1.close() #to change file access modes

write_txt(name = 'enron_train_wordRNN', data = short_data_consize)

## LSTM Model

In [16]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

text = read_file('enron_train_wordRNN.txt')
line = text.split("\n")
token_lst = [x.split() for x in line]

In [17]:
def generate_seq(tokens):
    train_len = 3+1
    text_sequences = []
    for i in range(train_len,len(tokens)):
        seq = tokens[i-train_len:i]
        text_sequences.append(seq)
    return text_sequences

seq_lst = [generate_seq(x) for x in token_lst]

In [18]:
flatten = [item for sublist in token_lst for item in sublist]

sequences = {}
count = 1
for i in range(len(flatten)):
    if flatten[i] not in sequences:
        sequences[flatten[i]] = count
        count += 1

In [19]:
flatten_seq = [item for sublist in seq_lst for item in sublist]

In [20]:
flatten_seq[40:50]

[['meetings', 'think', 'it', 'would'],
 ['think', 'it', 'would', 'be'],
 ['it', 'would', 'be', 'more'],
 ['would', 'be', 'more', 'to'],
 ['be', 'more', 'to', 'try'],
 ['more', 'to', 'try', 'and'],
 ['to', 'try', 'and', 'discussions'],
 ['try', 'and', 'discussions', 'across'],
 ['and', 'discussions', 'across', 'the'],
 ['discussions', 'across', 'the', 'different']]

In [21]:
text_sequences = flatten_seq
train_len = 3+1

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)

import numpy as np
n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [30]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]

train_targets = to_categorical(train_targets, num_classes=vocabulary_size+1)
seq_len = train_inputs.shape[1]
train_inputs.shape

def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len,input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(vocabulary_size,activation='softmax'))
    opt_adam = Adam(lr=0.001)
    #You can simply pass 'adam' to optimizer in compile method. Default learning rate 0.001
    #But here we are using adam optimzer from optimizer class to change the LR.
    model.compile(loss='categorical_crossentropy',optimizer=opt_adam,metrics=['accuracy'])
    model.summary()
    return model

In [24]:
model = create_model(vocabulary_size+1,seq_len)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 3)              7833      
_________________________________________________________________
lstm (LSTM)                  (None, 3, 50)             10800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 2611)              133161    
Total params: 174,544
Trainable params: 174,544
Non-trainable params: 0
_________________________________________________________________


In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

path = 'models/enron_wordRNN_0714.h5'
checkpoint = ModelCheckpoint(path, monitor='loss', verbose=1, save_best_only=True, mode='min')
stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=3)
model.fit(train_inputs,train_targets,batch_size=128,epochs=500,verbose=1,callbacks=[checkpoint, stopping])

Train on 225490 samples
Epoch 1/500
225280/225490 [============================>.] - ETA: 0s - loss: 3.7081 - accuracy: 0.3149
Epoch 00001: loss improved from inf to 3.70814, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 16s 70us/sample - loss: 3.7081 - accuracy: 0.3150
Epoch 2/500
225152/225490 [============================>.] - ETA: 0s - loss: 3.6880 - accuracy: 0.3156
Epoch 00002: loss improved from 3.70814 to 3.68798, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 16s 69us/sample - loss: 3.6880 - accuracy: 0.3156
Epoch 3/500
224768/225490 [============================>.] - ETA: 0s - loss: 3.6787 - accuracy: 0.3163
Epoch 00003: loss improved from 3.68798 to 3.67821, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 16s 71us/sample - loss: 3.6782 - accuracy: 0.3163
Epoch 4/500
225408/225490 [============================>.] - ETA: 0s - loss: 3.6717

225152/225490 [============================>.] - ETA: 0s - loss: 3.4387 - accuracy: 0.3513
Epoch 00055: loss improved from 3.44181 to 3.43867, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 17s 77us/sample - loss: 3.4387 - accuracy: 0.3513
Epoch 56/500
224896/225490 [============================>.] - ETA: 0s - loss: 3.4347 - accuracy: 0.3520 ETA: 0s - loss: 3.4343 - accuracy: 
Epoch 00056: loss improved from 3.43867 to 3.43449, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 17s 77us/sample - loss: 3.4345 - accuracy: 0.3520
Epoch 57/500
224768/225490 [============================>.] - ETA: 0s - loss: 3.4305 - accuracy: 0.3525
Epoch 00057: loss improved from 3.43449 to 3.43116, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 17s 77us/sample - loss: 3.4312 - accuracy: 0.3524
Epoch 58/500
225280/225490 [============================>.] - ETA: 0s - loss:

225280/225490 [============================>.] - ETA: 0s - loss: 3.3049 - accuracy: 0.3717
Epoch 00109: loss improved from 3.30587 to 3.30498, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 18s 79us/sample - loss: 3.3050 - accuracy: 0.3717
Epoch 110/500
225408/225490 [============================>.] - ETA: 0s - loss: 3.3029 - accuracy: 0.3721
Epoch 00110: loss improved from 3.30498 to 3.30321, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 18s 79us/sample - loss: 3.3032 - accuracy: 0.3721
Epoch 111/500
224896/225490 [============================>.] - ETA: 0s - loss: 3.3003 - accuracy: 0.3724
Epoch 00111: loss improved from 3.30321 to 3.30025, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 18s 81us/sample - loss: 3.3003 - accuracy: 0.3725
Epoch 112/500
225152/225490 [============================>.] - ETA: 0s - loss: 3.2987 - accuracy: 0.3730
Epoch 

Epoch 137/500
225280/225490 [============================>.] - ETA: 0s - loss: 3.2573 - accuracy: 0.3784
Epoch 00137: loss did not improve from 3.25715
225490/225490 [==============================] - 17s 77us/sample - loss: 3.2572 - accuracy: 0.3784
Epoch 138/500
225280/225490 [============================>.] - ETA: 0s - loss: 3.2546 - accuracy: 0.3800
Epoch 00138: loss improved from 3.25715 to 3.25485, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 17s 78us/sample - loss: 3.2548 - accuracy: 0.3800
Epoch 139/500
225152/225490 [============================>.] - ETA: 0s - loss: 3.2543 - accuracy: 0.3796
Epoch 00139: loss improved from 3.25485 to 3.25445, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 18s 78us/sample - loss: 3.2544 - accuracy: 0.3796
Epoch 140/500
225024/225490 [============================>.] - ETA: 0s - loss: 3.2511 - accuracy: 0.3804
Epoch 00140: loss improved from 3.25445 to

Epoch 165/500
225152/225490 [============================>.] - ETA: 0s - loss: 3.2200 - accuracy: 0.3843
Epoch 00165: loss did not improve from 3.21909
225490/225490 [==============================] - 17s 77us/sample - loss: 3.2197 - accuracy: 0.3843
Epoch 166/500
225152/225490 [============================>.] - ETA: 0s - loss: 3.2177 - accuracy: 0.3852
Epoch 00166: loss improved from 3.21909 to 3.21796, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 17s 77us/sample - loss: 3.2180 - accuracy: 0.3852
Epoch 167/500
224768/225490 [============================>.] - ETA: 0s - loss: 3.2151 - accuracy: 0.3856 ETA
Epoch 00167: loss improved from 3.21796 to 3.21527, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 17s 77us/sample - loss: 3.2153 - accuracy: 0.3856
Epoch 168/500
225024/225490 [============================>.] - ETA: 0s - loss: 3.2133 - accuracy: 0.3856
Epoch 00168: loss improved from 3.2152

Epoch 222/500
224896/225490 [============================>.] - ETA: 0s - loss: 3.1583 - accuracy: 0.3935
Epoch 00222: loss improved from 3.15935 to 3.15810, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 18s 79us/sample - loss: 3.1581 - accuracy: 0.3936
Epoch 223/500
225024/225490 [============================>.] - ETA: 0s - loss: 3.1577 - accuracy: 0.3938
Epoch 00223: loss improved from 3.15810 to 3.15751, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 18s 79us/sample - loss: 3.1575 - accuracy: 0.3938
Epoch 224/500
224896/225490 [============================>.] - ETA: 0s - loss: 3.1586 - accuracy: 0.3932
Epoch 00224: loss did not improve from 3.15751
225490/225490 [==============================] - 18s 79us/sample - loss: 3.1591 - accuracy: 0.3931
Epoch 225/500
225408/225490 [============================>.] - ETA: 0s - loss: 3.1556 - accuracy: 0.3937
Epoch 00225: loss improved from 3.15751 to

Epoch 251/500
225280/225490 [============================>.] - ETA: 0s - loss: 3.1383 - accuracy: 0.3961
Epoch 00251: loss did not improve from 3.13473
225490/225490 [==============================] - 17s 76us/sample - loss: 3.1386 - accuracy: 0.3961
Epoch 252/500
225280/225490 [============================>.] - ETA: 0s - loss: 3.1351 - accuracy: 0.3970
Epoch 00252: loss did not improve from 3.13473
225490/225490 [==============================] - 17s 75us/sample - loss: 3.1354 - accuracy: 0.3969
Epoch 253/500
225152/225490 [============================>.] - ETA: 0s - loss: 3.1341 - accuracy: 0.3975
Epoch 00253: loss improved from 3.13473 to 3.13427, saving model to models/enron_wordRNN_0714.h5
225490/225490 [==============================] - 18s 78us/sample - loss: 3.1343 - accuracy: 0.3975
Epoch 254/500
225408/225490 [============================>.] - ETA: 0s - loss: 3.1334 - accuracy: 0.3974
Epoch 00254: loss improved from 3.13427 to 3.13342, saving model to models/enron_wordRNN_071

In [33]:
model.save(path)

from pickle import dump
dump(tokenizer,open('models/enron_tokenizer_wordRNN_0714','wb'))  

## Prediction 

### Large dataset

Training results:

- Epoch 273/500 loss: 3.1200 - accuracy: 0.3993
- Each epoch takes about 80-100 seconds on GPU (RTX 2070 SUPER)

In [34]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pickle import load

model = load_model('models/enron_wordRNN_0714.h5')
tokenizer = load(open('models/enron_tokenizer_wordRNN_0714','rb'))
seq_len = 3 
def gen_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len,truncating='pre')
        pred_word_ind = model.predict_classes(pad_encoded,verbose=0)[0]
        
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' '+pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)

In [36]:
print('\n\n===>Enter --exit to exit from the program')
while True:
    seed_text  = input('Enter string: ')
    if seed_text.lower() == '--exit':
        break
    else:
        out = gen_text(model, tokenizer, seq_len=seq_len, seed_text=seed_text, num_gen_words=5)
        print('Output: '+seed_text+' '+out)
        print()



===>Enter --exit to exit from the program
Enter string: discussions
Output: discussions as the arcade and the

Enter string: meetings
Output: meetings as hot for the restructuring

Enter string: presentation
Output: presentation financials to strong long and

Enter string: stock
Output: stock to the immediate value ability

Enter string: finance
Output: finance as trades and resource rental

Enter string: --exit


In [43]:
print('\n\n===>Enter --exit to exit from the program')
while True:
    seed_text  = input('Enter string: ')
    if seed_text.lower() == '--exit':
        break
    else:
        out = gen_text(model, tokenizer, seq_len=seq_len, seed_text=seed_text, num_gen_words=5)
        print('Output: '+seed_text+' '+out)
        print()



===>Enter --exit to exit from the program
Enter string: business plan is
Output: business plan is the largest term must be

Enter string: round table discussion
Output: round table discussion of the value of the

Enter string: business meetings are
Output: business meetings are expected to rise on the

Enter string: business meetings
Output: business meetings as exodus documents to manage

Enter string: business
Output: business and potentially mexico and introducing

Enter string: --exit


### Smaller dataset

Training results:

- Epoch 500/500 loss: 1.4459 - accuracy: 0.6336
- Each epoch takes about 80-100 seconds on GPU (RTX 2070 SUPER)

In [5]:
model = load_model('models/outlook_wordRNN_0713.h5')
tokenizer = load(open('models/outlook_tokenizer_wordRNN_0713','rb'))
seq_len = 3 

print('\n\n===>Enter --exit to exit from the program')
while True:
    seed_text  = input('Enter string: ')
    if seed_text.lower() == '--exit':
        break
    else:
        out = gen_text(model, tokenizer, seq_len=seq_len, seed_text=seed_text, num_gen_words=5)
        print('Output: '+seed_text+' '+out)
        print()



===>Enter --exit to exit from the program
Enter string: meeting
Output: meeting meeting multi singapore singapore designer

Enter string: discussions
Output: discussions have to one last not

Enter string: presentation
Output: presentation have to one last not

Enter string: stock
Output: stock have to one last not

Enter string: finance
Output: finance have to one last not

Enter string: --exit


In [6]:
print('\n\n===>Enter --exit to exit from the program')
while True:
    seed_text  = input('Enter string: ')
    if seed_text.lower() == '--exit':
        break
    else:
        out = gen_text(model, tokenizer, seq_len=seq_len, seed_text=seed_text, num_gen_words=5)
        print('Output: '+seed_text+' '+out)
        print()



===>Enter --exit to exit from the program
Enter string: business plan is
Output: business plan is have until be only the

Enter string: round table discussion
Output: round table discussion have to one last not

Enter string: business meetings are
Output: business meetings are have data designer protect this

Enter string: business meetings
Output: business meetings have to one last not

Enter string: --exit


## Conclusion

- A larger dataset is able to predict the next word better
- The predictions are not very accurate